This is a benchmark for Autogluon Forecasting task.  
Assume that we have two COV19 dataset, train_data.csv and test_data.csv in the following format:  

|Date|ConfirmedCases|name|
| ------ | ------ | ------ |
|2020-01-22|0.0|Afghanistan_|
|2020-01-23|0.0|Afghanistan_|
|2020-01-24|0.0|Afghanistan_|
|2020-01-25|0.0|Afghanistan_|
|2020-01-26|0.0|Afghanistan_|

The comfirmedcases here is the cummulative comfirmed cases up to that date in a certain country.

In [1]:
# Load dataset
from autogluon.forecasting.task.forecasting.forecasting import Forecasting as task

train_data = task.Dataset(file_path="https://autogluon.s3-us-west-2.amazonaws.com/datasets/CovidTimeSeries/train.csv")
test_data = task.Dataset(file_path="https://autogluon.s3-us-west-2.amazonaws.com/datasets/CovidTimeSeries/test.csv")
prediction_length = 19
eval_metric = "mean_wQuantileLoss"

/Users/yixiaxia/.pyenv/versions/3.7.7/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/yixiaxia/.pyenv/versions/3.7.7/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/yixiaxia/.pyenv/versions/3.7.7/lib/python3.7/site-packages/gluonts/evaluation/_base.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sized


In [2]:
train_data.head()

/Users/yixiaxia/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Date,ConfirmedCases,name
0,2020-01-22,0.0,Afghanistan_
1,2020-01-23,0.0,Afghanistan_
2,2020-01-24,0.0,Afghanistan_
3,2020-01-25,0.0,Afghanistan_
4,2020-01-26,0.0,Afghanistan_


## Directly using GluonTS to do forecasting

In [3]:
import pandas as pd


def rebuild_tabular(X, time_column, target_column, index_column=None):
    """
    X: dataframe to rebuild, should have the form of:
    >>> X
      index_column time_column  target_column
    0            A  2020-01-22              1
    1            A  2020-01-23              2
    2            A  2020-01-24              3
    3            B  2020-01-22              1
    4            B  2020-01-23              2
    5            B  2020-01-24              3
    6            C  2020-01-22              1
    7            C  2020-01-23              2
    8            C  2020-01-24              3

    index_column: time series index, in the above example, there are three ts: A, B, C,
                  if index_column is None, we will assume that the dataset contains only one time series

    time_column: time of a data, in the form "YYYY-MM-DD HH:MM:SS", we are assuming that each time series contains the same time sequence,
                 and the freq in each time series does not change.

    target_column: values used for prediction, integers.

    output:
    a new dataframe in the form that each line contains a time series
    transformed example would be:
    >>> X
          index_column  2020-01-22  2020-01-23  2020-01-24
    0            A           1           2           3
    1            C           1           2           3
    2            B           1           2           3

    """
    if index_column is None:
        X = X[[time_column, target_column]]
        X["index_column"] = ["time_series" for i in range(X.shape[0])]
        index_column = "index_column"
    time_list = sorted(list(set(X[time_column])))
    freq = pd.infer_freq(time_list)
    if freq is None:
        raise ValueError("Freq cannot be inferred. Check your dataset.")

    def reshape_dataframe(df):
        """
        for each time occurs in the dataset, we select the target value corresponding to
        each time series, and use dataframe.pivot() to convert it to one column, where the column name is the
        time, each row is the corresponding target value for each time series.
        """
        df = df.sort_values(by=index_column)
        data_dic = {index_column: sorted(list(set(df[index_column])))}

        for time in time_list:
            tmp = df[df[time_column] == time][[index_column, time_column, target_column]]
            tmp = tmp.pivot(index=index_column, columns=time_column, values=target_column)
            tmp_values = tmp[time].values
            data_dic[time] = tmp_values
        return pd.DataFrame(data_dic)

    X = reshape_dataframe(X)
    return X

In [4]:
rebuilt_train = rebuild_tabular(train_data, time_column="Date", target_column="ConfirmedCases", index_column="name")
rebuilt_test = rebuild_tabular(test_data, time_column="Date", target_column="ConfirmedCases", index_column="name")

In [5]:
rebuilt_train.head()

,name,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02
0,Afghanistan_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,74.0,84.0,94.0,110.0,110.0,120.0,170.0,174.0,237.0,273.0
1,Albania_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,123.0,146.0,174.0,186.0,197.0,212.0,223.0,243.0,259.0,277.0
2,Algeria_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,264.0,302.0,367.0,409.0,454.0,511.0,584.0,716.0,847.0,986.0
3,Andorra_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,164.0,188.0,224.0,267.0,308.0,334.0,370.0,376.0,390.0,428.0
4,Angola_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,4.0,4.0,5.0,7.0,7.0,7.0,8.0,8.0


In [6]:
rebuilt_test.head()

,name,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21
0,Afghanistan_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,607.0,665.0,714.0,784.0,840.0,906.0,933.0,996.0,1026.0,1092.0
1,Albania_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,446.0,467.0,475.0,494.0,518.0,539.0,548.0,562.0,584.0,609.0
2,Algeria_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1914.0,1983.0,2070.0,2160.0,2268.0,2418.0,2534.0,2629.0,2718.0,2811.0
3,Andorra_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,638.0,646.0,659.0,673.0,673.0,696.0,704.0,713.0,717.0,717.0
4,Angola_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.0,19.0,19.0,19.0,19.0,19.0,24.0,24.0,24.0,24.0


Train-Val Split

In [7]:
rebuilt_val = rebuilt_train.copy()
rebuilt_train = rebuilt_train.iloc[:, :-prediction_length]

Then transform the rebuilt tabular into GluonTS Listdata

In [8]:
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset

def create_gluonts_data(df, index_column):
    index = df[index_column]
    target = df.drop(index_column, axis=1)
    target_values = target.values
    date_list = target.columns
    freq = pd.infer_freq(date_list)
    data = [
        {
            FieldName.TARGET: target,
            FieldName.START: pd.Timestamp(date_list[0], freq=freq),
            FieldName.ITEM_ID: item_id
        }
        for (target, item_id) in zip(target_values, index)
    ]
    return ListDataset(data, freq)

In [9]:
gluonts_train_data = create_gluonts_data(rebuilt_train, "name")
gluonts_val_data = create_gluonts_data(rebuilt_val, "name")
gluonts_test_data = create_gluonts_data(rebuilt_test, "name")

Train and evaluate

In [10]:
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.trainer import Trainer

params = {
    "prediction_length": 19,
    "freq": "D",
    "num_batches_per_epoch": 10,
    "epochs": 100,
    "quantiles": [0.1, 0.5, 0.9]
}
model = MQCNNEstimator.from_hyperparameters(**params)

predictor = model.train(gluonts_train_data)

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 10/10 [00:00<00:00, 36.10it/s, epoch=98/100, avg_epoch_loss=12.6]


100%|██████████| 10/10 [00:00<00:00, 35.40it/s, epoch=100/100, avg_epoch_loss=16.5]


In [11]:
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from tqdm import tqdm


forecast_it, ts_it = make_evaluation_predictions(dataset=gluonts_test_data,
                                                 predictor=predictor,
                                                 num_samples=100)
forecasts, tss = list(forecast_it), list(ts_it)
# forecasts, tss = list(tqdm(forecast_it, total=len(gluonts_val_data))), list(tqdm(ts_it, total=len(gluonts_val_data)))
# print(forecasts[0], tss[0])
evaluator = Evaluator()
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(tss))
print(agg_metrics[eval_metric])

/Users/yixiaxia/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Running evaluation: 100%|██████████| 313/313 [00:00<00:00, 8443.04it/s]


0.11829968436159917


Get prediction results for 20 days after the dataset

In [12]:
result_dict = {}
predicted_targets = list(predictor.predict(gluonts_test_data))
index = sorted(list(set(train_data["name"])))

quantiles = [0.5]
for i in range(len(index)):
    tmp_dict = {}
    for quantile in quantiles:
        tmp_dict[quantile] = predicted_targets[i].quantile(str(quantile))
    df = pd.DataFrame(tmp_dict)
    df.index = pd.date_range(start=predicted_targets[i].start_date,
                             periods=prediction_length,
                             freq="D")
    result_dict[index[i]] = df
    
print(result_dict["Afghanistan_"])

                    0.5
2020-04-22  1019.208862
2020-04-23   982.361755
2020-04-24   992.114441
2020-04-25   989.922974
2020-04-26  1028.132568
2020-04-27  1018.237427
2020-04-28  1023.813538
2020-04-29  1018.235596
2020-04-30  1042.546631
2020-05-01  1052.428711
2020-05-02  1030.224365
2020-05-03  1052.249512
2020-05-04   998.043213
2020-05-05  1051.093262
2020-05-06  1059.899902
2020-05-07  1032.044312
2020-05-08  1049.196777
2020-05-09  1030.370483
2020-05-10  1037.513794


## Using AutoGluon To do this forecasting

In [ ]:
from autogluon.forecasting.task.forecasting.forecasting import Forecasting as task

import autogluon.core as ag

# change this to specify search strategy, can try bayesopt, random, or skopt
searcher_type = "random"
# change this to specify eval metric, one of ["MASE", "MAPE", "sMAPE", "mean_wQuantileLoss"]
eval_metric = "mean_wQuantileLoss"

predictor = task.fit(train_data=train_data,
                     prediction_length=19,
                     index_column="name",
                     target_column="ConfirmedCases",
                     time_column="Date",
                     hyperparameter_tune=True,
                     hyperparameters={"MQCNN": {'context_length': ag.Int(10, 20),
                                                'epochs': 100,
                                                "num_batches_per_epoch": 10}},
                     search_strategy=searcher_type,
                     eval_metric=eval_metric,
                     num_trials=3)

print(predictor.leaderboard())

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.



100%|██████████| 10/10 [00:00<00:00, 37.26it/s, epoch=1/100, avg_epoch_loss=263]

100%|██████████| 10/10 [00:00<00:00, 37.17it/s, epoch=2/100, avg_epoch_loss=257]

  0%|          | 0/10 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.



100%|██████████| 10/10 [00:00<00:00, 35.72it/s, epoch=3/100, avg_epoch_loss=249]

100%|██████████| 10/10 [00:00<00:00, 36.34it/s, epoch=1/100, avg_epoch_loss=269]

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 33.98it/s, epoch=4/100, avg_epoch_loss=237]


learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.



100%|██████████| 10/10 [00:00<00:00, 35.06it/s, epoch=2/100, avg_epoch_loss=262]

100%|██████████| 10/10 [00:00<00:00, 27.84it/s, epoch=5/100, avg_epoch_loss=214]

100%|██████████| 10/10 [00:00<00:00, 31.65it/s, epoch=3/100, avg_epoch_loss=255]

  0%|          | 0/10 [00:00<?, ?it/s]

Evaluate the model

In [20]:
print(predictor.evaluate(test_data))

Running evaluation: 100%|██████████| 313/313 [00:00<00:00, 6015.17it/s]


0.33742358997324406


Get prediction results for 20 days after the dataset

In [21]:
predictions = predictor.predict(test_data, quantiles=[0.1, 0.5, 0.9])
print(predictions['Afghanistan_'])

                    0.1          0.5          0.9
2020-04-22   927.118286  1074.994629  1180.039185
2020-04-23   973.479370  1103.497070  1194.868652
2020-04-24   889.371399  1129.917725  1198.302734
2020-04-25   925.780457  1123.990601  1192.414551
2020-04-26   999.970154  1137.439575  1308.415527
2020-04-27   941.977539  1131.409058  1304.347168
2020-04-28   861.965149  1146.861694  1289.805908
2020-04-29  1027.356079  1148.889038  1272.243164
2020-04-30  1034.995483  1176.468262  1376.541992
2020-05-01   914.351135  1165.311646  1336.263428
2020-05-02   971.587769  1192.618164  1476.304688
2020-05-03   935.683350  1172.089600  1326.004395
2020-05-04   995.432861  1160.596069  1347.831787
2020-05-05   957.176208  1172.679565  1318.155029
2020-05-06  1005.680969  1179.221680  1343.417725
2020-05-07   954.338379  1170.545898  1309.378540
2020-05-08   883.415100  1183.472290  1419.175415
2020-05-09   960.499695  1205.509155  1483.166138
2020-05-10   994.224792  1190.478394  1407.080322
